In [ ]:
import seaborn as sns

from fau_colors import register_cmaps
register_cmaps()

palette = sns.color_palette("faculties_light")
palette = palette[:1] + palette[2:]
sns.set_palette(palette)

# Set presentation context
sns.set_context("notebook", font_scale=1)
sns.set_style("whitegrid")
sns.set_style("ticks")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os 
os.chdir('../')
sns.set_context("paper")

# Table for residuals

In [ ]:
### TABLE for residual forces and moments, first cluster
models = {
    "MRI": {
        "suffix": "_MRI"
    },
    "scaled": {
        "suffix": ''
    },
    "SIPP": {
        "suffix": '-SBP-HIT_body'
    },
    "S-gen": {
        "suffix": '_SIPP_generic'
    },
    "addB": {
        "suffix": '-addB'
    },
    "addB+SIPP": {
        "suffix": '-addB-SBP-HIT_body'
    },
    "addB+S-gen": {
        "suffix": '-addB-SBP-SGEN_None'
    },
}
#models = {
#    "S-gen": models["S-gen"],
#    "S-gen_misgendered": {
#        "suffix": '_SIPP_generic_misgendered'
#    },
#}

for model in models:
    models[model]['F'] = {'Fx': [], 'Fy': [], 'Fz': [], "F_norm": []}
    models[model]['M'] = {'Mx': [], 'My': [], 'Mz': [], "M_norm": []}

collfunc = lambda x: x.values # np.mean(np.abs(x))
pipelines = [['HIT', 'body']]#, ['InsideHumans', 'skin']] #['HIT', 'scan']]
participants = ['P01','P02','P03','P04','P05','P06']
mass_list = [72.31, 73.35, 64.49, 89.36, 87.63,44.91]
bodyheight = [1.67,1.77,1.65,1.77,1.79,1.44]
result_df = pd.DataFrame(columns=['participant', 'trial','MSK Model', 'dimension', 'value','value_rel'])
for participant,mass,height in zip(participants, mass_list, bodyheight):
    for trial in range(1,7):
        if (participant == 'P06' and trial == 5): continue
        if (participant == 'P01' and trial > 2): continue
        for model in models:
            if model in ['MRI', 'S-gen', 'S-gen_misgendered']:
                id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)
            else:
                id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial_{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)
            # discard the first and last 1000 rows for filtering artifacts
            id1 = id1.iloc[1000:-1000] 

            for dim, name in zip(['pelvis_tx_force', 'pelvis_ty_force', 'pelvis_tz_force', 'pelvis_tilt_moment', 'pelvis_rotation_moment', 'pelvis_list_moment'], ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']):
                if dim in id1.columns:
                    if name in ['Fx', 'Fy', 'Fz']:
                        models[model]['F'][name].append(collfunc(id1[dim])/mass/9.81*100)
                    if name in ['Mx', 'My', 'Mz']:
                        models[model]['M'][name].append(collfunc(id1[dim])/mass/9.81*100/ height)
                else:
                    raise ValueError(f"Dimension {dim} not found in {participant} Trial {trial} {model} data.")
            models[model]['F']['F_norm'].append(np.linalg.norm([id1['pelvis_tx_force'], id1['pelvis_ty_force'], id1['pelvis_tz_force']], axis=0)/mass/9.81*100)
            models[model]['M']['M_norm'].append(np.linalg.norm([id1['pelvis_tilt_moment'], id1['pelvis_list_moment'], id1['pelvis_rotation_moment']], axis=0)/mass/9.81*100/ height)
        #result_df.loc[len(result_df)] = [participant, trial, 'addB-IK', 'f_norm', np.mean(np.linalg.norm([id2[dim] for dim in ['pelvis_tx_force', 'pelvis_ty_force', 'pelvis_tz_force']], axis=0))/mass/9.81, 0]
        #result_df.loc[len(result_df)] = [participant, trial, 'addB-IK', 'm_norm', np.mean(np.linalg.norm([id2[dim] for dim in ['pelvis_tilt_moment', 'pelvis_list_moment', 'pelvis_rotation_moment']], axis=0))/mass/9.81, 0]

In [ ]:
# Write a nice latex table from the results
# HEADER:  & \multicolumn{4}{|c|}{\bf Forces} & \multicolumn{4}{|c|}{\bf Moments}\\ \hline
#   \bf Model & $\mathbf{F}_x~[\si{\newton}]$ & $\mathbf{F}_y~[\si{\newton}]$ & $\mathbf{F}_z~[\si{\newton}]$ & $||\mathbf{F}||~[\si{\newton}]$ & $\mathbf{M}_x~[\si{\newton\meter}]$ & $\mathbf{M}_y~[\si{\newton\meter}]$ & $\mathbf{M}_z~[\si{\newton\meter}]$ & $||\mathbf{M}||~[\si{\newton\meter}]$\\ \thickhline
# Write entries as $value \pm std$ for each model, highlight the best value in each column
for model in models:
    for dim in ['F', 'M']:
        for name in ['x', 'y', 'z', '_norm']:
            models[model][dim][dim+name+'value'] = np.mean(np.abs(np.concatenate(models[model][dim][dim+name])))
            models[model][dim][dim+name+'std'] = np.std(np.abs(np.concatenate(models[model][dim][dim+name])))
# Get the best values for each dimension
best_values_01 = {}
best_values_02 = {}
for dim in ['F', 'M']:
    for name in ['x', 'y', 'z', '_norm']:
        best_values_01[dim + name] = min(models[model][dim][dim+name+'value'] for model in models if not model.startswith('addB'))
        best_values_02[dim + name] = min(models[model][dim][dim+name+'value'] for model in models if model.startswith('addB'))

# Print it nicely in LaTeX format
for model in models:
    if model == 'addB':
        print("\\hline")
    print(f"{model} ", end='')
    for dim in ['F', 'M']:
        for name in ['x', 'y', 'z', '_norm']:
            value = models[model][dim][dim+name+'value']
            std = models[model][dim][dim+name+'std']
            if value == (best_values_01[dim + name] if not model.startswith('addB') else best_values_02[dim + name]):
                print(f"& $\\mathbf{{{value:.2f} \\pm {std:.1f}}}$  ", end='')
            else:
                print(f"& ${value:.2f} \pm {std:.1f}$ ", end='')
    print("\\\\")  # End of row

In [ ]:
# Step 1: Build a list of small DataFrames, one per (model, dimension+name).
dfs = []
dfs_595 = []  # For the 5-95 percentile filtered data
for model_name, dims in models.items():
    for dim in ['F', 'M']:
        for suffix in ['x', 'y', 'z', '_norm']:
            key = dim + suffix
            arr = np.concatenate(dims[dim][key])  # shape: (N_values,)
            # Only use the 5-95 percentile range
            arr 
            # Create a DataFrame of length N_values in one shot
            temp_df = pd.DataFrame({
                'MSK Model': model_name,
                'dimension': key,
                'value': arr
            })
            dfs.append(temp_df)
            temp_df = pd.DataFrame({
                'MSK Model': model_name,
                'dimension': key ,
                'value': arr[(arr >= np.percentile(arr, 5)) & (arr <= np.percentile(arr, 95))]
            })
            dfs_595.append(temp_df)
            print(f"Added {model_name} {key} with {len(arr)} values, filtered to {len(temp_df)} values in 5-95 percentile range, with the maximum value {np.percentile(arr, 95):.2f}")

# Step 2: Concatenate all of them at once
boxplot_df = pd.concat(dfs, ignore_index=True)
boxplot_df_595 = pd.concat(dfs_595, ignore_index=True)

In [ ]:
boxplot_df.head()

In [ ]:
dims = ['F_norm', 'M_norm', 'F_norm', 'M_norm']
hues = ['n', 'n', 'opt', 'opt']

figure = plt.figure(figsize=(5.2, 4.2))
for i, dim, hue in zip(range(4), dims, hues):
    bx2 = boxplot_df[boxplot_df.dimension == dim].copy()
    if hue == 'n':
        models_ = ['scaled', 'SIPP', 'S-gen',"MRI"]
    else:
        models_ = ['addB', 'addB+SIPP', 'addB+S-gen']

    bx2 = bx2[bx2['MSK Model'].isin(models_)]
    ax = figure.add_subplot(2, 2, i + 1)
    if i == 0:
        ax_ = ax
    if i == 2:
        ax__ = ax
    sns.boxenplot(
        data=bx2,
        #x='dimension',
        y='value',
        x='MSK Model',
        showfliers=True,
        ax=ax,
        hue_order=models_,
        order=models_,
        palette=palette,
        #showmeans=True,meanprops={'marker':'^','markerfacecolor':'grey','markeredgecolor':'black','markersize':'4'}
    )
    if hue == 'opt':
        # Replace xticks with the model names
        ax.set_xticklabels(['addB', 'addB\n+SIPP', 'addB\n+S-gen'], rotation=0, ha='center')


    group_means = bx2.groupby('MSK Model')['value'].mean()

    if dim == 'F_norm':
        ax.set_ylabel(r'$F_{norm}~[\text{BW%}]$')
    else:
        ax.set_ylabel(r'$M_{norm}~[\text{BWBH%}]$')

    ax.set_xlabel('')

    for j, model in enumerate(models_):
        mean_val = group_means[model]
        ax.scatter(
            x=j,  # position in order of models_
            y=mean_val,
            color='black',
            marker='^',
            s=50,
            zorder=10,
            label='_nolegend_'  # Prevent extra legend entry
        )
        ax.scatter(
            x=j,  # position in order of models_
            y=mean_val,
            color='grey',
            marker='^',
            s= 10,
            zorder=10,
            label='_nolegend_'  # Prevent extra legend entry
        )
    upper_lim = [21,7,15,6]
    ax.set_ylim(0, upper_lim[i%2])
    
plt.tight_layout()
plt.text(-0.22, 1.05, 'a)', transform=ax_.transAxes, fontsize=12, fontweight='bold', va='top', ha='right')
plt.text(-0.22, 1.05, 'b)', transform=ax__.transAxes, fontsize=12, fontweight='bold', va='top', ha='right')
plt.savefig('../../Topics/Publications/mrigait_empkins/figures/residuals.pdf', bbox_inches='tight')

# Plot for metabolics

In [ ]:
### TABLE for residual forces and moments, first cluster
models = {
    "MRI": {
        "suffix": "_MRI",
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled": {
        "suffix": '',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP": {
        "suffix": '-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "S-gen": {
        "suffix": '_SIPP_generic',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "addB": {
        "suffix": '-addB',
        "ik_folder": lambda x: f"results/{x}/gait/addB-with_phys/generic/IK"
    },
    "addB+SIPP": {
        "suffix": '-addB-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-with_phys/S0{x[-1]}_HIT/IK"
    },
    "addB+S-gen": {
        "suffix": '-addB-SBP-SGEN_None',
        "ik_folder": lambda x: f"results/{x}/gait/addB-with_phys/S0{x[-1]}_SGEN/IK"
    },
}

def KIMR15(M,q):
    mq = M*q
    mq_pos = mq.copy()
    mq_pos[mq_pos < 0] = 0
    mq_neg = mq.copy()
    mq_neg[mq_neg > 0] = 0
    h_m, h_sl_s, h_sl_l, q_cc = 0.054, 0.283, 1.423, 0.004
    #h_m, h_sl_s, h_sl_l, q_cc = 0, 0, 1, 0
    return np.mean(h_m * np.max(np.abs(q)) * np.abs(M) + h_sl_s * mq_pos - h_sl_l * mq_neg + q_cc * np.max(M*q) + M*q)

for model in models:
    models[model]['F'] = {'Fx': [], 'Fy': [], 'Fz': [], "F_norm": []}
    models[model]['M'] = {'Mx': [], 'My': [], 'Mz': [], "M_norm": []}

collfunc = lambda x: x.values # np.mean(np.abs(x))
pipelines = [['HIT', 'body']]#, ['InsideHumans', 'skin']] #['HIT', 'scan']]
participants = ['P01','P02','P03','P04','P05','P06']
for model in models:
    models[model]['MEE'] = {f"Trial{trial}": {p: [] for p in participants} for trial in range(1, 7)}

mass_list = [72.31, 73.35, 64.49, 89.36, 87.63,44.91]
bodyheight = [1.67,1.77,1.65,1.77,1.79,1.44]
speeds = [1.3, 0.8, 1.3, 0.8, 1.3, 0.8] 
result_df = pd.DataFrame(columns=['participant', 'trial','MSK Model', 'dimension', 'value','value_rel'])
for participant,mass,height in zip(female_subset, mass_list, bodyheight):
    for trial, speed in zip(range(1,7), speeds):
        if (participant == 'P06' and trial == 5): continue
        if (participant == 'P01' and trial > 2): continue
        for model in models:
            if model in ['MRI', 'S-gen']:
                id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)
            else:
                id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial_{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)

            ik = pd.read_csv(f"{models[model]['ik_folder'](participant)}/Trial{trial}_filtered.mot", sep='\t', skiprows=10)

            if len(id1) != len(ik):
                print(f"Warning: Length mismatch for {participant} Trial {trial} {model}: ID {len(id1)} vs IK {len(ik)}")
                continue
            MEE = 0
            for col in ik.columns:
                if col.startswith('pelvis') or col.startswith('time'):
                    continue
                q = np.gradient(ik[col])*100
                M = id1[col+'_moment']
                MEE += KIMR15(M, q) / mass / speed
            models[model]['MEE'][f"Trial{trial}"][participant].append(MEE)


In [ ]:
fig = plt.figure(figsize=(5.2, 4))
#fig = plt.figure(figsize=(5, 7.5))
# Create a DataFrame with the trial names, participants, models, and MEE values
result_df = pd.DataFrame(columns=['participant', 'trial', 'MSK Model', 'MEE'])
for participant, mass, height, speed in zip(participants, mass_list, bodyheight, speeds):
    for trial in range(1, 7):
        if (participant == 'P06' and trial == 5): continue
        if (participant == 'P01' and trial > 2): continue
        for model in models:
            result_df.loc[len(result_df)] = [participant, trial, model, models[model]['MEE'][f"Trial{trial}"][participant]]
result_df['MEE'] = result_df['MEE'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
#sns.boxplot(x='MSK Model', y='MEE', data=result_df, palette=sns.color_palette("faculties_light"))
ax = fig.add_subplot(211)
x = 'trial'
hue = 'MSK Model'
#hue = 'participant'
# Only show the follwing models: 
models_of_interest = ['scaled', 'SIPP', 'S-gen', 'MRI']
result_df = result_df[result_df['MSK Model'].isin(models_of_interest)]

sns.boxplot(result_df, x=x, y='MEE', hue=hue,showmeans=True,meanprops={'marker':'^','markerfacecolor':'grey','markeredgecolor':'black','markersize':'4'}, hue_order=models_of_interest)
# Y grid
# Plot the mean for every column
# Rename the xticks
plt.xticks(labels=["1.3 m/s\nlevel", "0.8 m/s\nlevel", "1.3 m/s\nincline", "0.8 m/s\nincline", "1.3 m/s\ndecline", "0.8 m/s\ndecline"], ticks=[0, 1, 2, 3, 4, 5])
plt.legend(loc='upper left', ncol=2, title='MSK Model',bbox_to_anchor=(-0.007, 1.02), labelspacing=0.3, columnspacing=0.3, handletextpad=0.3)
plt.ylabel('Metabolic Cost\n[J/kg/m]')  # set the y-label
plt.xlabel('')
ax_ = fig.add_subplot(212)
# Print mean and std for each model
for model in models:
    mean = np.mean(result_df[result_df['MSK Model'] == model]['MEE'])
    std = np.std(result_df[result_df['MSK Model'] == model]['MEE'])
    if not np.isnan(mean):
        print(f"{model}: {mean:.2f} ± {std:.2f}")

# Legend position top left with two columns

# Create a DataFrame with the trial names, participants, models, and MEE values
result_df = pd.DataFrame(columns=['participant', 'trial', 'MSK Model', 'MEE'])
for participant, mass, height, speed in zip(participants, mass_list, bodyheight, speeds):
    for trial in range(1, 7):
        if (participant == 'P06' and trial == 5): continue
        if (participant == 'P01' and trial > 2): continue
        for model in models:
            if model.startswith('addB+'):
                m = model.split('+')[0][0] +'+'+ model.split('+')[1]
            else:
                m = model
            result_df.loc[len(result_df)] = [participant, trial, m, models[model]['MEE'][f"Trial{trial}"][participant]]
result_df['MEE'] = result_df['MEE'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
#sns.boxplot(x='MSK Model', y='MEE', data=result_df, palette=sns.color_palette("faculties_light"))

x = 'trial'
hue = 'MSK Model'
#hue = 'participant'
# Only show the follwing models: 
models_of_interest = ['addB', 'a+SIPP', 'a+S-gen']
result_df = result_df[result_df['MSK Model'].isin(models_of_interest)]

sns.boxplot(result_df, x=x, y='MEE', hue=hue,showmeans=True,meanprops={'marker':'^','markerfacecolor':'grey','markeredgecolor':'black','markersize':'4'})
# Y grid
# Plot the mean for every column
# Rename the xticks
plt.xticks(labels=["1.3 m/s\nlevel", "0.8 m/s\nlevel", "1.3 m/s\nincline", "0.8 m/s\nincline", "1.3 m/s\ndecline", "0.8 m/s\ndecline"], ticks=[0, 1, 2, 3, 4, 5])
plt.legend(loc='upper left', ncol=2, title='MSK Model', bbox_to_anchor=(-0.007, 1.02), labelspacing=0.3, columnspacing=0.3, handletextpad=0.3)
plt.ylabel('Metabolic Cost\n[J/kg/m]')  # set the y-label
plt.xlabel('')
# set the legend names manually

# Print mean and std for each model
for model in np.unique(result_df['MSK Model']):
    mean = np.mean(result_df[result_df['MSK Model'] == model]['MEE'])
    std = np.std(result_df[result_df['MSK Model'] == model]['MEE'])
    if not np.isnan(mean):
        print(f"{model}: {mean:.2f} ± {std:.2f}")




plt.tight_layout()
# Write a bold a and b in the top left corner over each subplot
plt.text(-0.05, 1.05, 'b)', transform=ax_.transAxes, fontsize=12, fontweight='bold', va='top', ha='right')
plt.text(-0.05, 1.05, 'a)', transform=ax.transAxes, fontsize=12, fontweight='bold', va='top', ha='right')
plt.savefig('../../Topics/Publications/mrigait_empkins/figures/MEE_result_female.pdf', bbox_inches='tight')

# Plot for joint moments (practical difference)

In [ ]:
### TABLE for residual forces and moments, first cluster
trial = 1
models = {
    "MRI": {
        "suffix": "_MRI",
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled": {
        "suffix": '',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP": {
        "suffix": '-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "S-gen": {
        "suffix": '_SIPP_generic',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "addB": {
        "suffix": '-addB',
        "ik_folder": lambda x: f"results/{x}/gait/addB-with_phys/generic/IK"
    },
    "addB+SIPP": {
        "suffix": '-addB-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-with_phys/S0{x[-1]}_HIT/IK"
    },
    "addB+S-gen": {
        "suffix": '-addB-SBP-SGEN_None',
        "ik_folder": lambda x: f"results/{x}/gait/addB-with_phys/S0{x[-1]}_SGEN/IK"
    },
}
for model in models:
    models[model]['F'] = {'Fx': [], 'Fy': [], 'Fz': [], "F_norm": []}
    models[model]['M'] = {'Mx': [], 'My': [], 'Mz': [], "M_norm": []}

collfunc = lambda x: x.values # np.mean(np.abs(x))
pipelines = [['HIT', 'body']]#, ['InsideHumans', 'skin']] #['HIT', 'scan']]
participants = ['P01','P02','P03','P04','P05','P06']
for model in models:
    models[model]['MEE'] = {f"Trial{trial}": {p: {} for p in participants} for trial in range(1, 7)}

mass_list = [72.31, 73.35, 64.49, 89.36, 87.63,44.91]
bodyheight = [1.67,1.77,1.65,1.77,1.79,1.44]
speeds = [1.3, 0.8, 1.3, 0.8, 1.3, 0.8] 
result_df = pd.DataFrame(columns=['participant', 'trial','MSK Model', 'dimension', 'value','value_rel'])


### Ciómbine all the joint moments into a nice table
#mass = mass_list[participants.index(participant)]
#height = bodyheight[participants.index(participant)]
#speed = speeds[trial]



joint_moments = ['hip_flexion_r_moment', 'knee_angle_l_moment', 'ankle_angle_r_moment', "lumbar_extension_moment", "hip_adduction_r_moment", "knee_angle_r_moment", "hip_rotation_r_moment", "lumbar_rotation_moment", "lumbar_bending_moment", "ankle_angle_r_moment"]
res_dict = {p: {m:{j: [] for j in joint_moments} for m in models} for p in participants}
res_dict_std = {p: {m:{j: [] for j in joint_moments} for m in models} for p in participants}


##if (participant == 'P06' and trial == 5): raise ValueError(f"Participant {participant} Trial {trial} not available.")
#if (participant == 'P01' and trial > 2): raise ValueError(f"Participant {participant} Trial {trial} not available.")
for participant in participants:
    for model in models:
        if model in ['MRI', 'S-gen']:
            id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)
        else:
            id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial_{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)
        #ik = pd.read_csv(f"{models[model]['ik_folder'](participant)}/Trial{trial}_filtered.mot", sep='\t', skiprows=10)


        grf_path = f"data/{participant}/gait/Trial{trial}_filtered.mot"
        grf = pd.read_csv(grf_path, sep='\t', skiprows=5)   

        grf_y = grf['1_ground_force_vy'].to_numpy().copy()
        grf_y[grf_y<30] = 0
        grf_y[grf_y>0] = 1

        # Find all rising edges
        rising_edges = np.where(np.diff(grf_y) == 1)[0]
        # Plot the data from the first rising edge to the next falling edge

        grf_y = grf['1_ground_force_vy'].to_numpy()
        gc_list = []    
        for i in range(len(rising_edges)-1):
            start = rising_edges[i]
            end = rising_edges[i+1]
            gc_list.append((start, end))

        # plot the lengths of each gaait cycle
        mean_gc_len = np.mean([end-start for start, end in gc_list])
        std_gc_len = np.std([end-start for start, end in gc_list])
        gc_filtered = []
        for start, end in gc_list:
            if abs(end-start-mean_gc_len) < std_gc_len:
                gc_filtered.append((start, end))

        for dim in joint_moments:
            # loadID 
            # Grab all gait cycles
            gc_list = []
            for gc in gc_filtered:
                start, end = gc
                # Get the index of the first and last gait cycle
                id1_gc = id1[dim].iloc[start:end].to_numpy().copy()
                # interpolate to 100 points
                id1_gc = np.interp(np.linspace(0, len(id1_gc)-1, 100), np.arange(len(id1_gc)), id1_gc)
                gc_list.append(id1_gc)
            mean_id1 = np.mean(gc_list, axis=0)
            std_id1 = np.std(gc_list, axis=0)
            res_dict[participant][model][dim].append(mean_id1)
            res_dict_std[participant][model][dim].append(std_id1)


In [ ]:
figure = plt.figure(figsize=(7.5, 4))
pp = "P01"
#figure = plt.figure(figsize=(15,15))
model_order = ['scaled', 'SIPP', 'S-gen', 'MRI','addB', 'addB+SIPP', 'addB+S-gen']
for i, dim in enumerate(["hip_flexion_r_moment", "hip_adduction_r_moment", "ankle_angle_r_moment", "lumbar_extension_moment"]):
    reset_here = True
    ax = figure.add_subplot(2, 2, i+1)
    for model in model_order:
        mean = np.mean(res_dict[pp][model][dim])
        std = np.std(res_dict[pp][model][dim])
        if mean == 0 and std == 0:
            continue
        if model.startswith('addB+'):
            m = model.split('+')[0][0] +'+'+ model.split('+')[1]
        else:
            m = model
        if not model.startswith('addB'):
            plt.plot(np.linspace(0, 100, 100), res_dict[pp][model][dim][0], label=m)
            plt.fill_between(np.linspace(0, 100, 100), res_dict[pp][model][dim][0] - res_dict_std[pp][model][dim][0], res_dict[pp][model][dim][0] + res_dict_std[pp][model][dim][0], alpha=0.2)
        else:
            if reset_here:
                # Reset the colorpropagation
                reset_here = False
                plt.gca().set_prop_cycle(None)
            plt.plot(np.linspace(0, 100, 100), res_dict[pp][model][dim][0], linestyle='--')
            #plt.fill_between(np.linspace(0, 100, 100), res_dict[pp][model][dim][0] - res_dict_std[pp][model][dim][0], res_dict[pp][model][dim][0] + res_dict_std[pp][model][dim][0], alpha=0.2)
        
    plt.title(dim.replace('_', ' ').capitalize())
    plt.xlabel('Gait Cycle [%]')
    plt.ylabel('Moment [Nm]')
    if dim == "hip_flexion_r_moment":
        plt.legend(loc='lower center', ncol=2, title='MSK Model', labelspacing=0.3, columnspacing=0.3, handletextpad=0.3)
    plt.xlim(0, 100)
    plt.grid()
    labels = {"hip_flexion_r_moment": "Hip Flexion Moment", "knee_angle_l_moment": "Contralateral Knee Flexion Moment", "ankle_angle_r_moment": "Ankle Flexion Moment", "lumbar_extension_moment": "Lumbar Extension Moment", "hip_adduction_r_moment": "Hip Adduction Moment", 
    plt.title(labels[dim])
    #plt.yscale('log')
plt.tight_layout()
#plt.savefig(f'../../Topics/Publications/mrigait_empkins/figures/joint_moments_{pp}.pdf', bbox_inches='tight')

In [ ]:
### By changing "dim" in this loop, you can plot different joint moments for all participants

figure = plt.figure(figsize=(7.5, 6))
pp = "P01"
# set font size to 20
#figure = plt.figure(figsize=(15,15))
model_order = ['scaled', 'SIPP', 'S-gen', 'MRI','addB', 'addB+SIPP', 'addB+S-gen']
dim = "ankle_angle_r_moment"
participants = ['P01', 'P02', 'P03', 'P04', 'P05', 'P06']
for i, pp in enumerate(participants):
    reset_here = True
    ax = figure.add_subplot(3, 2, i+1)
    for model in model_order:
        mean = np.mean(res_dict[pp][model][dim])
        std = np.std(res_dict[pp][model][dim])
        if mean == 0 and std == 0:
            continue
        if model.startswith('addB+'):
            m = model.split('+')[0][0] +'+'+ model.split('+')[1]
        else:
            m = model
        legend_entries = {'scaled': 'scaled', 'SIPP': 'shape-based', 'MRI': 'MRI-based'}
        if not model.startswith('addB'):

            plt.plot(np.linspace(0, 100, 100), res_dict[pp][model][dim][0], label=legend_entries.get(model, m), linewidth=2)
            plt.fill_between(np.linspace(0, 100, 100), res_dict[pp][model][dim][0] - res_dict_std[pp][model][dim][0], res_dict[pp][model][dim][0] + res_dict_std[pp][model][dim][0], alpha=0.2)
        else:
            if reset_here:
                # Reset the colorpropagation
                reset_here = False
                plt.gca().set_prop_cycle(None)
            plt.plot(np.linspace(0, 100, 100), res_dict[pp][model][dim][0], linestyle='--')
            #plt.fill_between(np.linspace(0, 100, 100), res_dict[pp][model][dim][0] - res_dict_std[pp][model][dim][0], res_dict[pp][model][dim][0] + res_dict_std[pp][model][dim][0], alpha=0.2)
        
    plt.title(dim.replace('_', ' ').capitalize())
    plt.xlabel('Gait Cycle [%]')
    plt.ylabel('Moment [Nm]')
    if pp == "P06":
        plt.legend(loc='lower left', ncol=1, labelspacing=0.3, 
                   columnspacing=0.2, handletextpad=0.3,
                     bbox_to_anchor=(0.1, -0.045))
    plt.xlim(0, 100)
    plt.grid()
    labels = {"hip_flexion_r_moment": "Hip Flexion Moment", "knee_angle_l_moment": "Contralateral Knee Flexion Moment", "ankle_angle_r_moment": "Ankle Flexion Moment", "lumbar_extension_moment": "Lumbar Extension Moment", "hip_adduction_r_moment": "Hip Adduction Moment", "hip_rotation_r_moment": "Hip Rotation Moment", "lumbar_rotation_moment": "Lumbar Rotation Moment", "lumbar_bending_moment": "Lumbar Bending Moment", "ankle_angle_l_moment": "Contralateral Ankle Flexion Moment"}
    plt.title(labels[dim])
    plt.xticks()
    plt.yticks()
# remove the legend from the last subplot
#plt.legend(title='MSK Model', labelspacing=0.3, columnspacing=0.3, handletextpad=0.3, bbox_to_anchor=(0.5, 0.54))
#.remove()
#plt.title('')
# remove grid

# Hide grid lines

    #plt.yscale('log')
plt.tight_layout()
plt.savefig(f'../../Topics/Publications/mrigait_empkins/figures/{dim}_all_pps.pdf', bbox_inches='tight')

# A summary of differences between SIPP-generic and generic

In [ ]:
layers = ['hand', 'ulna', 'humerus', 'HAT', 'femur', 'tibia', 'foot']
def stack_masses(df):
    new_df = pd.DataFrame(columns=['name', 'mass'])
    for layer in layers:
        if layer == 'hand':
            mass = df[df['name'].str.startswith('hand')]['mass'].sum()
        elif layer == 'foot':
            mass = df[df['name'].str.startswith('foot')]['mass'].sum()
        elif layer == 'HAT':
            mass = df[df['name'].str.startswith('HAT')]['mass'].sum()
        elif layer == "ulna":
            mass = df[df['name'].str.startswith('ulna')]['mass'].sum() + df[df['name'].str.startswith('radius')]['mass'].sum()
        elif layer == "humerus":
            mass = df[df['name'].str.startswith('humerus')]['mass'].sum()
        elif layer == "femur":
            mass = df[df['name'].str.startswith('femur')]['mass'].sum()
        elif layer == "tibia":
            mass = df[df['name'].str.startswith('tibia')]['mass'].sum() + df[df['name'].str.startswith('patella')]['mass'].sum()
        new_df.loc[len(new_df)] = [layer, mass]
    # normalize the masses to sum to 1
    new_df['mass'] = new_df['mass'] / new_df['mass'].sum()
    return new_df

In [ ]:
import pandas as pd

def generate_flow_df(df1, df2, source_col='mass'):
    """
    Generate a long-format flow DataFrame from a wide-format source-target table.

    Parameters:
    - df: pandas DataFrame with columns [source_col, source_prefix, target_prefix]
      where source_col has layer names, and columns named source_prefix and target_prefix have counts.
    - source_col: the column name for layers (default 'Source').
    - source_prefix: prefix for source nodes (default 'a').
    - target_prefix: prefix for target nodes (default 'b').

    This function uses the first character of the layer name as its abbreviation.
    """
    layers = df1['name'].tolist()
    a_counts = df1[source_col].tolist()
    b_counts = df2[source_col].tolist()

    flows = []
    j = 0  # index for target layers
    target_remaining = b_counts[j]

    for i, source_count in enumerate(a_counts):
        source_remaining = source_count

        while source_remaining > 0 and j < len(b_counts):
            allocation = min(source_remaining, target_remaining)
            from_layer = f"a_{layers[i]}"
            to_layer = f"b_{layers[j]}"
            if i!= j:
                flows.append({
                    'from_layer': from_layer,
                    'to_layer': to_layer,
                    'value': allocation
                })

            source_remaining -= allocation
            target_remaining -= allocation

            if target_remaining == 0:
                j += 1
                if j < len(b_counts):
                    target_remaining = b_counts[j]
    
    # Set all remaining source counts to zero
    for i in range(len(a_counts)):
        for j in range(len(b_counts)):
            if i != j:
                from_layer = f"a_{layers[i]}"
                to_layer   = f"b_{layers[j]}"
                # Check if the flow already exists
                if not any(flow['from_layer'] == from_layer and flow['to_layer'] == to_layer for flow in flows):
                    flows.append({
                        'from_layer': from_layer,
                        'to_layer':   to_layer,
                        'value':      0
                    })
        

    return pd.DataFrame(flows)


In [ ]:
# Plotting function
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from matplotlib.path import Path
import matplotlib.patches as patches

def plot_sankey_from_flow_df(flow_df):
    flows = [
        (row['from_layer'], row['to_layer'], row['value'])
        for _, row in flow_df.iterrows()
    ]

    def draw_s_curve_arrow(ax, start, end, lw=1.0):
        """Draws an S-curve from start to end with a cubic Bezier curve."""
        sx, sy = start
        ex, ey = end
        mid_x = (sx + ex) / 2
        curve_offset = (ey - sy) * 0.1  # adjust for curvature

        # Control points to make the 'S' shape
        control1 = (mid_x, sy + curve_offset)
        control2 = (mid_x, ey - curve_offset)

        path = Path([start, control1, control2, end],
                    [Path.MOVETO, Path.CURVE4, Path.CURVE4, Path.CURVE4])

        patch = patches.FancyArrowPatch(path=path,
                                         arrowstyle='->',
                                         color='black',
                                         lw=lw,
                                         mutation_scale=10)
        ax.add_patch(patch)

    # Plot setup

    positions = {
    'a_hand': (-1, 2.13),
    'a_ulna': (-1, 1.93),
    'a_humerus': (-1, 1.73),
    'a_HAT': (-1, 1.15),
    'a_femur': (-1, 0.7),
    'a_tibia': (-1, 0.3),
    'a_foot': (-1, 0.02),
    'b_hand': (1, 2.13), 
    'b_ulna': (1, 1.93),
    'b_humerus': (1, 1.73),
    'b_HAT': (1, 1.15),
    'b_femur': (1, 0.7),
    'b_tibia': (1, 0.3),
    'b_foot': (1, 0.02)
    }   

    # Draw nodes
    for node, (x, y) in positions.items():
        continue
        ax.plot(x, y, 'o', markersize=10, color='skyblue')
        ax.text(x, y + 0.1, node, ha='center')

    # Draw arrows with thickness proportional to flow
    for src, tgt, weight in flows:
        start = positions[src]
        end = positions[tgt]
        linewidth = weight * 100  # scale thickness
        draw_s_curve_arrow(ax, start, end, lw=linewidth)

    


    

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_obj_faces_2d(obj_path, omit_dim='z', offset=0.0, dir=1, offset_y=0.0):
    def load_obj(filepath):
        verts = []
        faces = []

        with open(filepath, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue
                if parts[0] == 'v':
                    # Vertex line: v x y z
                    verts.append([float(parts[1]), float(parts[2]), float(parts[3])])
                elif parts[0] == 'f':
                    # Face line: f i j k ... (OBJ is 1-based index)
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                    faces.append(face)

        # Convert to DataFrame for vertices (optional but requested)
        df_verts = pd.DataFrame(verts, columns=['x', 'y', 'z'])
        return df_verts, faces

    def plot_faces_2d(df_verts, faces, omit_dim='z'):
        # Map omit_dim to index
        dim_map = {'x':0, 'y':1, 'z':2}
        omit_idx = dim_map[omit_dim]

        # Prepare figure
        # For each face, plot polygon ignoring one dimension
        for face in faces:
            pts = df_verts.loc[face].values  # Nx3 array
            # Remove omitted dimension
            pts_2d = [ (pt[(omit_idx+1)%3], pt[(omit_idx+2)%3]) for pt in pts ]
            pts_2d_2 = [pt[omit_idx] for pt in pts]
            if np.any(np.array(pts_2d_2)<0):
                # If any point has a negative value in the omitted dimension, skip this face
                continue
            pts_2d.append(pts_2d[0])  # Close the polygon


            xs, ys = zip(*pts_2d)
            plt.plot(np.array(ys)*dir+offset, np.array(xs)+offset_y, 'k-', linewidth=0.1)

        plt.gca().set_aspect('equal')

    # Usage:
    df_verts, faces = load_obj(obj_path)
    plot_faces_2d(df_verts, faces, omit_dim='x')



In [ ]:
# Parameterization differences, generate flow table
from src.opensim import opensim_utils
male_subset = ['P02','P04', 'P05']
female_subset = ['P01', 'P03', 'P06']
fig = plt.figure(figsize=(5.2,5.2))
for i, model in enumerate(['SIPP', 'SIPP-generic', 'addBiomechanics', 'MRI']):
    flow_dfs = []
    ax = fig.add_subplot(2,2,i+1)
    for participant in female_subset + male_subset:
        p = participant
        scaled = f"results/{p}/gait/addB-with_phys/generic/Models/match_markers_but_ignore_physics.osim"
        addB = f"results/{p}/gait/addB-with_phys/generic/Models/match_markers_and_physics.osim"
        SIPP = f"results/{p}/bodyhull/bsipw_shoes/HIT_body_loose_w_skel.osim"
        SIPP_generic = f"results/{p}/gait/ID_results/SMPL_generic_scale.osim"
        MRI = f"results/{p}/MRI/osim_models/MRI.osim"

        gen_mod = opensim_utils.get_model(scaled)
        addB_mod = opensim_utils.get_model(addB)
        SIPP_mod = opensim_utils.get_model(SIPP)
        smpl_gen_mod = opensim_utils.get_model(SIPP_generic)
        mri_mod = opensim_utils.get_model(MRI)

        gen_df = opensim_utils.get_bsip_dataframe_with_feet_and_HAT(gen_mod)
        addB_df = opensim_utils.get_bsip_dataframe_with_feet_and_HAT(addB_mod)
        SIPP_df = opensim_utils.get_bsip_dataframe_with_feet_and_HAT(SIPP_mod)
        smpl_gen_df = opensim_utils.get_bsip_dataframe_with_feet_and_HAT(smpl_gen_mod)
        mri_df = opensim_utils.get_bsip_dataframe_with_feet_and_HAT(mri_mod)

        gen_df = stack_masses(gen_df)
        if model == 'addBiomechanics':
            other_df = stack_masses(addB_df)
        elif model == 'SIPP':
            other_df = stack_masses(SIPP_df)
        elif model == 'SIPP-generic':
            other_df = stack_masses(smpl_gen_df)
        elif model == 'MRI':
            other_df = stack_masses(mri_df)

        flow_dfs.append(generate_flow_df(gen_df, other_df, source_col='mass'))
    flow_dfs = pd.concat(flow_dfs, ignore_index=True)
    # Get mean of a for each flow
    flow_dfs_mean = flow_dfs.groupby(['from_layer', 'to_layer']).mean().reset_index()
    plot_sankey_from_flow_df(flow_dfs_mean)
    plot_obj_faces_2d(f"notebooks/generic_raised_arms.obj", omit_dim='x', offset=1.0, offset_y = 1.21, dir=-1)
    plot_obj_faces_2d(f"notebooks/generic_raised_arms.obj", omit_dim='x', offset=-1.0, offset_y=1.21, dir=1)
    # Final formatting
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-0.5, 2.5)
    ax.axis('off')
    ax.set_title(model)
    ax.set_rasterization_zorder(1)  # Elements below zorder 1 will be rasterized
#plt.tight_layout()
#fig.subplots_adjust(wspace=0., hspace=0.)
plt.savefig('../../Topics/Publications/mrigait_empkins/figures/mass_transfer.png', bbox_inches='tight', dpi=600)


# Sensitivity analysis, outputs

In [ ]:
### TABLE for residual forces and moments, first cluster
models = {
    "scaled": {
        "suffix": '',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP": {
        "suffix": '-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "InsideHumans": {
        "suffix": '-SBP-InsideHumans_skin',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SMPL": {
        "suffix": '-SBP-SMPL_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
}

def KIMR15(M,q):
    mq = M*q
    mq_pos = mq.copy()
    mq_pos[mq_pos < 0] = 0
    mq_neg = mq.copy()
    mq_neg[mq_neg > 0] = 0
    h_m, h_sl_s, h_sl_l, q_cc = 0.054, 0.283, 1.423, 0.004
    #h_m, h_sl_s, h_sl_l, q_cc = 0, 0, 1, 0
    return np.mean(h_m * np.max(np.abs(q)) * np.abs(M) + h_sl_s * mq_pos - h_sl_l * mq_neg + q_cc * np.max(M*q) + M*q)

for model in models:
    models[model]['F'] = {'Fx': [], 'Fy': [], 'Fz': [], "F_norm": []}
    models[model]['M'] = {'Mx': [], 'My': [], 'Mz': [], "M_norm": []}

collfunc = lambda x: x.values # np.mean(np.abs(x))
pipelines = [['HIT', 'body']]#, ['InsideHumans', 'skin']] #['HIT', 'scan']]
participants = ['P01','P02','P03','P04','P05','P06']
for model in models:
    models[model]['MEE'] = {f"Trial{trial}": {p: [] for p in participants} for trial in range(1, 7)}

mass_list = [72.31, 73.35, 64.49, 89.36, 87.63,44.91]
bodyheight = [1.67,1.77,1.65,1.77,1.79,1.44]
speeds = [1.3, 0.8, 1.3, 0.8, 1.3, 0.8] 
result_df = pd.DataFrame(columns=['participant', 'trial','MSK Model', 'dimension', 'value','value_rel'])
for participant,mass,height in zip(participants, mass_list, bodyheight):
    for trial, speed in zip(range(1,7), speeds):
        if (participant == 'P06' and trial == 5): continue
        if (participant == 'P01' and trial > 2): continue
        for model in models:
            if model in ['MRI', 'S-gen']:
                id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)
            else:
                id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial_{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)

            ik = pd.read_csv(f"{models[model]['ik_folder'](participant)}/Trial{trial}_filtered.mot", sep='\t', skiprows=10)

            if len(id1) != len(ik):
                print(f"Warning: Length mismatch for {participant} Trial {trial} {model}: ID {len(id1)} vs IK {len(ik)}")
                continue
            MEE = 0
            for col in ik.columns:
                if col.startswith('pelvis') or col.startswith('time'):
                    continue
                q = np.gradient(ik[col])*100
                M = id1[col+'_moment']
                MEE += KIMR15(M, q) / mass / speed
            models[model]['MEE'][f"Trial{trial}"][participant].append(MEE)


In [ ]:
fig = plt.figure(figsize=(5.2, 4))
#fig = plt.figure(figsize=(5, 7.5))
# Create a DataFrame with the trial names, participants, models, and MEE values
result_df = pd.DataFrame(columns=['participant', 'trial', 'MSK Model', 'MEE'])
for participant, mass, height, speed in zip(participants, mass_list, bodyheight, speeds):
    for trial in range(1, 7):
        if (participant == 'P06' and trial == 5): continue
        if (participant == 'P01' and trial > 2): continue
        for model in models:
            result_df.loc[len(result_df)] = [participant, trial, model, models[model]['MEE'][f"Trial{trial}"][participant]]
result_df['MEE'] = result_df['MEE'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
#sns.boxplot(x='MSK Model', y='MEE', data=result_df, palette=sns.color_palette("faculties_light"))
ax = fig.add_subplot(211)
x = 'trial'
hue = 'MSK Model'
#hue = 'participant'
# Only show the follwing models: 
#models_of_interest = ['scaled', 'SIPP', 'S-gen', 'MRI']
#result_df = result_df[result_df['MSK Model'].isin(models_of_interest)]

sns.boxplot(result_df, x=x, y='MEE', hue=hue,showmeans=True,meanprops={'marker':'^','markerfacecolor':'grey','markeredgecolor':'black','markersize':'4'})
# Y grid
# Plot the mean for every column
# Rename the xticks
plt.xticks(labels=["1.3 m/s\nlevel", "0.8 m/s\nlevel", "1.3 m/s\nincline", "0.8 m/s\nincline", "1.3 m/s\ndecline", "0.8 m/s\ndecline"], ticks=[0, 1, 2, 3, 4, 5])
plt.legend(loc='upper left', ncol=2, title='MSK Model',bbox_to_anchor=(-0.007, 1.02), labelspacing=0.3, columnspacing=0.3, handletextpad=0.3)
plt.ylabel('Metabolic Cost\n[J/kg/m]')  # set the y-label
plt.xlabel('')
ax_ = fig.add_subplot(212)
# Print mean and std for each model
for model in models:
    mean = np.mean(result_df[result_df['MSK Model'] == model]['MEE'])
    std = np.std(result_df[result_df['MSK Model'] == model]['MEE'])
    if not np.isnan(mean):
        print(f"{model}: {mean:.2f} ± {std:.2f}")

# Legend position top left with two columns

# Create a DataFrame with the trial names, participants, models, and MEE values
result_df = pd.DataFrame(columns=['participant', 'trial', 'MSK Model', 'MEE'])
for participant, mass, height, speed in zip(participants, mass_list, bodyheight, speeds):
    for trial in range(1, 7):
        if (participant == 'P06' and trial == 5): continue
        if (participant == 'P01' and trial > 2): continue
        for model in models:
            if model.startswith('addB+'):
                m = model.split('+')[0][0] +'+'+ model.split('+')[1]
            else:
                m = model
            result_df.loc[len(result_df)] = [participant, trial, m, models[model]['MEE'][f"Trial{trial}"][participant]]
#result_df['MEE'] = result_df['MEE'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
#sns.boxplot(x='MSK Model', y='MEE', data=result_df, palette=sns.color_palette("faculties_light"))

x = 'trial'
hue = 'MSK Model'
#hue = 'participant'
# Only show the follwing models: 
models_of_interest = ['addB', 'a+SIPP', 'a+S-gen']
result_df = result_df[result_df['MSK Model'].isin(models_of_interest)]

sns.boxplot(result_df, x=x, y='MEE', hue=hue,showmeans=True,meanprops={'marker':'^','markerfacecolor':'grey','markeredgecolor':'black','markersize':'4'})
# Y grid
# Plot the mean for every column
# Rename the xticks
plt.xticks(labels=["1.3 m/s\nlevel", "0.8 m/s\nlevel", "1.3 m/s\nincline", "0.8 m/s\nincline", "1.3 m/s\ndecline", "0.8 m/s\ndecline"], ticks=[0, 1, 2, 3, 4, 5])
plt.legend(loc='upper left', ncol=2, title='MSK Model', bbox_to_anchor=(-0.007, 1.02), labelspacing=0.3, columnspacing=0.3, handletextpad=0.3)
plt.ylabel('Metabolic Cost\n[J/kg/m]')  # set the y-label
plt.xlabel('')
# set the legend names manually

# Print mean and std for each model
for model in np.unique(result_df['MSK Model']):
    mean = np.mean(result_df[result_df['MSK Model'] == model]['MEE'])
    std = np.std(result_df[result_df['MSK Model'] == model]['MEE'])
    if not np.isnan(mean):
        print(f"{model}: {mean:.2f} ± {std:.2f}")




plt.tight_layout()
# Write a bold a and b in the top left corner over each subplot
plt.text(-0.05, 1.05, 'b)', transform=ax_.transAxes, fontsize=12, fontweight='bold', va='top', ha='right')
plt.text(-0.05, 1.05, 'a)', transform=ax.transAxes, fontsize=12, fontweight='bold', va='top', ha='right')
#plt.savefig('../../Topics/Publications/mrigait_empkins/figures/MEE_results.pdf', bbox_inches='tight')

In [ ]:
for model in models:
    models[model]['F'] = {'Fx': [], 'Fy': [], 'Fz': [], "F_norm": []}
    models[model]['M'] = {'Mx': [], 'My': [], 'Mz': [], "M_norm": []}

collfunc = lambda x: x.values # np.mean(np.abs(x))
pipelines = [['HIT', 'body']]#, ['InsideHumans', 'skin']] #['HIT', 'scan']]
participants = ['P01','P02','P03','P04','P05','P06']
mass_list = [72.31, 73.35, 64.49, 89.36, 87.63,44.91]
bodyheight = [1.67,1.77,1.65,1.77,1.79,1.44]
result_df = pd.DataFrame(columns=['participant', 'trial','MSK Model', 'dimension', 'value','value_rel'])
for participant,mass,height in zip(participants, mass_list, bodyheight):
    for trial in range(1,7):
        if (participant == 'P06' and trial == 5): continue
        if (participant == 'P01' and trial > 2): continue
        for model in models:
            if model in ['MRI', 'S-gen']:
                id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)
            else:
                id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial_{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)
            # discard the first and last 1000 rows for filtering artifacts
            id1 = id1.iloc[1000:-1000] 

            for dim, name in zip(['pelvis_tx_force', 'pelvis_ty_force', 'pelvis_tz_force', 'pelvis_tilt_moment', 'pelvis_rotation_moment', 'pelvis_list_moment'], ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']):
                if dim in id1.columns:
                    if name in ['Fx', 'Fy', 'Fz']:
                        models[model]['F'][name].append(collfunc(id1[dim])/mass/9.81*100)
                    if name in ['Mx', 'My', 'Mz']:
                        models[model]['M'][name].append(collfunc(id1[dim])/mass/9.81*100/ height)
                else:
                    raise ValueError(f"Dimension {dim} not found in {participant} Trial {trial} {model} data.")
            models[model]['F']['F_norm'].append(np.linalg.norm([id1['pelvis_tx_force'], id1['pelvis_ty_force'], id1['pelvis_tz_force']], axis=0)/mass/9.81*100)
            models[model]['M']['M_norm'].append(np.linalg.norm([id1['pelvis_tilt_moment'], id1['pelvis_list_moment'], id1['pelvis_rotation_moment']], axis=0)/mass/9.81*100/ height)
        #result_df.loc[len(result_df)] = [participant, trial, 'addB-IK', 'f_norm', np.mean(np.linalg.norm([id2[dim] for dim in ['pelvis_tx_force', 'pelvis_ty_force', 'pelvis_tz_force']], axis=0))/mass/9.81, 0]
        #result_df.loc[len(result_df)] = [participant, trial, 'addB-IK', 'm_norm', np.mean(np.linalg.norm([id2[dim] for dim in ['pelvis_tilt_moment', 'pelvis_list_moment', 'pelvis_rotation_moment']], axis=0))/mass/9.81, 0]

In [ ]:
# Write a nice latex table from the results
# HEADER:  & \multicolumn{4}{|c|}{\bf Forces} & \multicolumn{4}{|c|}{\bf Moments}\\ \hline
#   \bf Model & $\mathbf{F}_x~[\si{\newton}]$ & $\mathbf{F}_y~[\si{\newton}]$ & $\mathbf{F}_z~[\si{\newton}]$ & $||\mathbf{F}||~[\si{\newton}]$ & $\mathbf{M}_x~[\si{\newton\meter}]$ & $\mathbf{M}_y~[\si{\newton\meter}]$ & $\mathbf{M}_z~[\si{\newton\meter}]$ & $||\mathbf{M}||~[\si{\newton\meter}]$\\ \thickhline
# Write entries as $value \pm std$ for each model, highlight the best value in each column
for model in models:
    for dim in ['F', 'M']:
        for name in ['x', 'y', 'z', '_norm']:
            models[model][dim][dim+name+'value'] = np.mean(np.abs(np.concatenate(models[model][dim][dim+name])))
            models[model][dim][dim+name+'std'] = np.std(np.abs(np.concatenate(models[model][dim][dim+name])))
# Get the best values for each dimension
best_values_01 = {}
best_values_02 = {}
for dim in ['F', 'M']:
    for name in ['x', 'y', 'z', '_norm']:
        best_values_01[dim + name] = min(models[model][dim][dim+name+'value'] for model in models if not model.startswith('addB'))
        #best_values_02[dim + name] = min(models[model][dim][dim+name+'value'] for model in models if model.startswith('addB'))

# Print it nicely in LaTeX format
for model in models:
    if model == 'addB':
        print("\\hline")
    print(f"{model} ", end='')
    for dim in ['F', 'M']:
        for name in ['_norm']:
            value = models[model][dim][dim+name+'value']
            std = models[model][dim][dim+name+'std']
            if value == (best_values_01[dim + name] if not model.startswith('addB') else best_values_02[dim + name]):
                print(f" {value:.2f}  +- {std:.1f} ", end='|')
            else:
                print(f" {value:.2f}  +- {std:.1f} ", end='|')
    print()  # End of row

In [ ]:
models

In [ ]:
### TABLE for residual forces and moments, first cluster
models = {
    "scaled_f6": {
        "suffix": '_f6',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f6": {
        "suffix": '_f6-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled_f9": {
        "suffix": '_f9',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f9": {
        "suffix": '_f9-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled_f12": {
        "suffix": '_f12',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f12": {
        "suffix": '_f12-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled_f15": {
        "suffix": '_f15',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f15": {
        "suffix": '_f15-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled_f20": {
        "suffix": '_f20',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f20": {
        "suffix": '_f20-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled_f30": {
        "suffix": '_f30',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f30": {
        "suffix": '_f30-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },

}
for model in models:
    models[model]['F'] = {'Fx': [], 'Fy': [], 'Fz': [], "F_norm": []}
    models[model]['M'] = {'Mx': [], 'My': [], 'Mz': [], "M_norm": []}

collfunc = lambda x: x.values # np.mean(np.abs(x))
pipelines = [['HIT', 'body']]#, ['InsideHumans', 'skin']] #['HIT', 'scan']]
participants = ['P01','P02','P03','P04','P05','P06']
mass_list = [72.31, 73.35, 64.49, 89.36, 87.63,44.91]
bodyheight = [1.67,1.77,1.65,1.77,1.79,1.44]
result_df = pd.DataFrame(columns=['participant', 'trial','MSK Model', 'dimension', 'value','value_rel'])
for participant,mass,height in zip(participants, mass_list, bodyheight):
    for trial in range(1,7):
        if (participant == 'P06' and trial == 5): continue
        if (participant == 'P01' and trial > 2): continue
        for model in models:
            if model in ['MRI', 'S-gen']:
                id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)
            else:
                id1 = pd.read_csv(f"results/{participant}/gait/ID_results/Trial_{trial}{models[model]['suffix']}.sto", sep='\t', skiprows=6)
            # discard the first and last 1000 rows for filtering artifacts
            id1 = id1.iloc[1000:-1000] 

            for dim, name in zip(['pelvis_tx_force', 'pelvis_ty_force', 'pelvis_tz_force', 'pelvis_tilt_moment', 'pelvis_rotation_moment', 'pelvis_list_moment'], ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']):
                if dim in id1.columns:
                    if name in ['Fx', 'Fy', 'Fz']:
                        models[model]['F'][name].append(collfunc(id1[dim])/mass/9.81*100)
                    if name in ['Mx', 'My', 'Mz']:
                        models[model]['M'][name].append(collfunc(id1[dim])/mass/9.81*100/ height)
                else:
                    raise ValueError(f"Dimension {dim} not found in {participant} Trial {trial} {model} data.")
            models[model]['F']['F_norm'].append(np.linalg.norm([id1['pelvis_tx_force'], id1['pelvis_ty_force'], id1['pelvis_tz_force']], axis=0)/mass/9.81*100)
            models[model]['M']['M_norm'].append(np.linalg.norm([id1['pelvis_tilt_moment'], id1['pelvis_list_moment'], id1['pelvis_rotation_moment']], axis=0)/mass/9.81*100/ height)

In [ ]:
# Write a nice latex table from the results
# HEADER:  & \multicolumn{4}{|c|}{\bf Forces} & \multicolumn{4}{|c|}{\bf Moments}\\ \hline
#   \bf Model & $\mathbf{F}_x~[\si{\newton}]$ & $\mathbf{F}_y~[\si{\newton}]$ & $\mathbf{F}_z~[\si{\newton}]$ & $||\mathbf{F}||~[\si{\newton}]$ & $\mathbf{M}_x~[\si{\newton\meter}]$ & $\mathbf{M}_y~[\si{\newton\meter}]$ & $\mathbf{M}_z~[\si{\newton\meter}]$ & $||\mathbf{M}||~[\si{\newton\meter}]$\\ \thickhline
# Write entries as $value \pm std$ for each model, highlight the best value in each column
for model in models:
    for dim in ['F', 'M']:
        for name in ['x', 'y', 'z', '_norm']:
            models[model][dim][dim+name+'value'] = np.mean(np.abs(np.concatenate(models[model][dim][dim+name])))
            models[model][dim][dim+name+'std'] = np.std(np.abs(np.concatenate(models[model][dim][dim+name])))
# Get the best values for each dimension
best_values_01 = {}
best_values_02 = {}
for dim in ['F', 'M']:
    for name in ['x', 'y', 'z', '_norm']:
        best_values_01[dim + name] = min(models[model][dim][dim+name+'value'] for model in models if not model.startswith('addB'))
        #best_values_02[dim + name] = min(models[model][dim][dim+name+'value'] for model in models if model.startswith('addB'))

# Print it nicely in LaTeX format
for model in models:
    if model == 'addB':
        print("\\hline")
    print(f"{model} ", end='')
    for dim in ['F', 'M']:
        for name in ['_norm']:
            value = models[model][dim][dim+name+'value']
            std = models[model][dim][dim+name+'std']
            if value == (best_values_01[dim + name] if not model.startswith('addB') else best_values_02[dim + name]):
                print(f" {value:.2f}  +- {std:.1f} ", end='|')
            else:
                print(f" {value:.2f}  +- {std:.1f} ", end='|')
    if model.startswith('scaled'):
        F_n1 = models[model]['F']['F_normvalue']
        M_n1 = models[model]['M']['M_normvalue']
        F_n2 = models[model.replace('scaled', 'SIPP')]['F']['F_normvalue']
        M_n2 = models[model.replace('scaled', 'SIPP')]['M']['M_normvalue']
        print()
        print('Difference for', model.split('_')[1], end=':\t')
        print(f" {100-100*F_n2/F_n1:.2f} | {(1-M_n2/M_n1)*100:.2f} ", end='|')
    print()  # End of row

In [ ]:
### TABLE for residual forces and moments, first cluster
models = {
    "scaled_f6": {
        "suffix": '_f6',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f6": {
        "suffix": '_f6-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled_f9": {
        "suffix": '_f9',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f9": {
        "suffix": '_f9-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled_f12": {
        "suffix": '_f12',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f12": {
        "suffix": '_f12-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled_f15": {
        "suffix": '_f15',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f15": {
        "suffix": '_f15-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled_f20": {
        "suffix": '_f20',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f20": {
        "suffix": '_f20-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "scaled_f30": {
        "suffix": '_f30',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
    "SIPP_f30": {
        "suffix": '_f30-SBP-HIT_body',
        "ik_folder": lambda x: f"results/{x}/gait/addB-ignore_phys/IK"
    },
}